In [ ]:
import sys
import random
import time

from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import numpy as np
# import geopandas as gpd
import matplotlib.pyplot as plt

import glob
import geopandas as gpd

from datetime import datetime

url = r'https://matokeo.necta.go.tz/csee2022/index.htm'

In [ ]:
pg = requests.get(url)
soup = bs(pg.content, 'html.parser')

In [ ]:
table = soup.find_all('table')[2]


In [ ]:
links = table.find_all('a')

# links
s_links = []
for link in links:
    lintext = link.text.strip()
    if not lintext.startswith('P'):
#         print(lintext)
        s_links.append(link)
print(s_links[110])

In [ ]:

n_retry = 0
def get_region(lin):
    page = requests.get(lin)
    sp = bs(page.content, 'html.parser')
    reg = sp.find_all('table')[4].find_all('p')[1].text
    return reg
        
    

def fetch_results(lin, name):
    
    
    global n_retry
    try:
        #Bad HTML Tables
        #Will read 'CIVICS' info as headers
        ts = pd.read_html(lin, header = 0)
    #         print(len(ts))
        df = ts[1]
        school_gpa = float(ts[2].iloc[0,1].split(' ')[0])

        df['centre_ref'] = name.split(' ')[0].strip()
        df['centre_name'] = ' '.join(name.split(' ')[1:])
        df['region'] = get_region(lin)
        n_retry = 0
        school_sat = float(ts[-3].columns[2])
        school_regd = float(ts[-3].columns[0])
        subjs_df = ts[-1]
        math_df = subjs_df[subjs_df.CIVICS == 'BASIC MATHEMATICS']
        math_gpa = float(math_df.iloc[0,-2])
        math_sat = float(math_df.iloc[0,3])
        
        swahili_df = subjs_df[subjs_df.CIVICS == 'KISWAHILI']
        swahili_gpa = float(swahili_df.iloc[0,-2])
        swahili_sat = float(swahili_df.iloc[0,3])
        
        school_data = {
            'name': name,
            'school_gpa': school_gpa,
            'math_gpa': math_gpa,
            'school_regd': school_regd,
            'school_sat': school_sat,
            'math_sat': math_sat,
            'swahili_gpa': swahili_gpa,
            'swahili_sat': swahili_sat,
        }
#         print(school_data)
        return ts[1], school_data
    
    except:
        n_retry += 1
        time.sleep(3)
        if n_retry <=5:
            print(f'Retrying .... #{n_retry}')
            fetch_results(lin,name)
            
        return None, {}
    
    

###==================START HERE ===============================
res_tables = []
schools_data = []
milestone = 110
count = milestone
step = 10
n_links = len(links[milestone:1000])

for j in range(milestone, 1000 ,step):
    print(datetime.now())
    print('*---*   ' * 10 )
    print(f'Working from {j+1} to {j+step}')
    print('-'*50)
    for link in s_links[j:j+step]:
        t = random.randint(0,2)
#         time.sleep(t)
        lintext = link.text.strip()

        href = r'https://matokeo.necta.go.tz/csee2022/' + link.get('href').replace('\\','/')

    #     if lintext.startswith('S'):
        print('{0}. Fetching {1}'.format(count+1, lintext))
        print('\t' + href)
        results, school_data = fetch_results(href, lintext)
        if school_data == {}:
            count+=1
            continue
        res_tables.append(results)
        schools_data.append(school_data)

        count +=1
    my_df = pd.concat(res_tables)
    my_df.to_csv(r'G:\My Drive\consultancy\CSEE2022 Analysis\2022\csee{}_{}.csv'.format(j+1,j+step))
    res_tables = []
    
    
    schools_df = pd.DataFrame(schools_data)
    schools_df.to_csv(r'G:\My Drive\consultancy\CSEE2022 Analysis\2022\schools{}_{}.csv'.format(j+1,j+step))
    schools_data = []
    

## Part II: Regional Analysis

In [ ]:
path = r'G:\My Drive\consultancy\CSEE2022 Analysis\2022' # use your path
all_files = glob.glob(path + "/*.csv")
# print(all_files)
li = []

for filename in all_files:
    fname = filename.split('\\')[-1]
#     print(fname)
    if fname.startswith('csee'):
        print('Processing ', fname)
        df = pd.read_csv(filename, index_col=None, header=0, usecols = ['CNO','SEX','AGGT','DIV','DETAILED SUBJECTS',
                                                                        'centre_ref','centre_name','region'])

        li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)
frame = frame.drop_duplicates()
frame

In [ ]:
tz_adm = gpd.read_file(r'tza_admbnda_adm1_20181019\tza_admbnda_adm1_20181019.shp')
tz_adm = tz_adm[['ADM1_EN',  'geometry']]
# tz_adm

In [ ]:
reg_results = frame[['CNO', 'AGGT', 'DIV', 'region']]
reg_results.loc[:, 'region'] = reg_results.region.str.title()
reg_results.replace({'region':{'Dar Es Salaam': 'Dar-es-salaam'}}, inplace = True)
# res_joined = reg_results.copy()
reg_results

In [ ]:

I_s = reg_results[reg_results.DIV == 'I']
div_I = I_s.groupby(['region','DIV']).count()
div_I.rename(columns = {'CNO':'Div_I'}, inplace = True)
div_I.sort_values(by = 'Div_I', inplace = True)

II_s = reg_results[reg_results.DIV == 'II']
div_II = II_s.groupby(['region','DIV']).count()
div_II.rename(columns = {'CNO':'Div_II'}, inplace = True)
div_II.sort_values(by = 'Div_II', inplace = True)

III_s = reg_results[reg_results.DIV == 'III']
div_III = III_s.groupby(['region','DIV']).count()
div_III.rename(columns = {'CNO':'Div_III'}, inplace = True)
div_III.sort_values(by = 'Div_III', inplace = True)

IV_s = reg_results[reg_results.DIV == 'IV']
div_IV = IV_s.groupby(['region','DIV']).count()
div_IV.rename(columns = {'CNO':'Div_IV'}, inplace = True)
div_IV.sort_values(by = 'Div_IV', inplace = True)


O_s = reg_results[reg_results.DIV == '0']
div_0 = O_s.groupby(['region','DIV']).count()
div_0.rename(columns = {'CNO':'Div_0'}, inplace = True)
div_0.sort_values(by = 'Div_0', inplace = True)

ABS_s = reg_results[reg_results.DIV == 'ABS']
ABS = ABS_s.groupby(['region','DIV']).count()
ABS.rename(columns = {'CNO':'ABS'}, inplace = True)
ABS.sort_values(by = 'ABS', inplace = True)

# ABS

In [ ]:
adm_results = tz_adm.merge(reg_results, left_on = 'ADM1_EN', right_on = 'region')
adm_results

In [ ]:
x = reg_results.groupby(['region','DIV']).count()[['CNO']]
div_result = x.unstack(level = -1)
div_result = div_result.fillna(0).astype(int)
div_result = div_result.droplevel(0, axis = 1)
div_result = div_result.drop(columns = ['*E','*R',  '*W', '-', 'ABS'])
div_result['total'] = div_result.sum(axis = 1)
div_result['I_pct'] = div_result.I/div_result.total*100
div_result['good_pct'] = (div_result.I+div_result.II+div_result.III)/div_result.total*100
div_result['low_pct'] = (div_result.IV+div_result['0'])/div_result.total*100
div_result = div_result.sort_values(by = 'low_pct', ascending = False)

div_result

In [ ]:
tz_results = tz_adm.merge(div_result, left_on = 'ADM1_EN', right_on = 'region', how = 'inner')
tz_results['centre'] = tz_results.geometry.centroid
zenj = ['Mjini Magharibi',  'Kusini Pemba', 'Kaskazini Pemba',
        'Kaskazini Unguja', 'Kusini Unguja', ]
zenj_results = tz_results[tz_results['ADM1_EN'].isin(zenj)]
tz_results =  tz_results[ ~(tz_results['ADM1_EN'].isin(zenj))].sort_values(by = 'low_pct', ascending = False)


In [ ]:
base = tz_results.plot(figsize = (10,10), column = 'good_pct', legend = False, cmap = 'Blues',
                k =5, edgecolor = '#ccc', linewidth = 1,
                 legend_kwds={'label': "% with Division I,II & III",'orientation': "vertical"}) #scheme = 'quantiles',
tz_results.centre.plot(ax = base, color = 'white', markersize = 2000, alpha = 0.5, edgecolor = 'cyan', lw = 1)
base.set_axis_off()
for x,y,value  in zip(tz_results.centre.x, tz_results.centre.y, tz_results.good_pct):
    base.annotate(round(value,1), xy=(x,y), xytext = (-12,-4), textcoords = 'offset points', fontsize = 12)
plt.title('CSEE 2021 Results\n% of School Candidates With Division I, II & III\n',fontdict = {'fontsize': 18,
        'fontweight': 'bold',
        'color': 'black',}
        )
base.margins(0)
# plt.savefig('good_pct-2021.png', dpi = 300)

In [ ]:
base = tz_results.plot(figsize = (15,15), column = 'low_pct', legend = False, cmap = 'Reds',
                k =5, edgecolor = '#ccc', linewidth = 1,
                       legend_kwds={'label': "% with Division I,II & III",'orientation': "horizontal"}) #scheme = 'quantiles',
tz_results.centre.plot(ax = base, color = 'white', markersize = 2000, alpha = 0.5, edgecolor = 'cyan', lw = 2)

for x,y,value  in zip(tz_results.centre.x, tz_results.centre.y, tz_results.low_pct):
    base.annotate(round(value,1), xy=(x,y), xytext = (-12,-4), textcoords = 'offset points', fontsize = 12)
base.set_axis_off()
plt.title('CSEE 2021 Results\n% of School Candidates With Division IV & 0\n',fontdict = {'fontsize': 18,
        'fontweight': 'bold',
        'color': 'black',}
        )
base.margins(0)
# plt.savefig('low_pct-2021.png', dpi = 300)

In [ ]:

fig, ax = plt.subplots(1,2, figsize = (15,10))

# zenj_results.plot(column = 'good_pct', ax = ax[0], figsize = (10,10), cmap = 'Blues')
zenj_results.plot(ax = ax[0], figsize = (15,15), column = 'good_pct', legend = False, cmap = 'Blues',
                k =5, edgecolor = '#ccc', linewidth = 1,
                 legend_kwds={'label': "% with Division I,II & III",'orientation': "horizontal"}) #scheme = 'quantiles',
zenj_results.centre.plot(ax = ax[0], color = 'white', markersize = 3000, alpha = 0.5, edgecolor = 'cyan', lw = 2)
for x,y,value  in zip(zenj_results.centre.x, zenj_results.centre.y, zenj_results.good_pct):
    ax[0].annotate(round(value,1), xy=(x,y), xytext = (-10,-4), textcoords = 'offset points', fontsize = 14)
# ax[0].margins(0)
ax[0].set_axis_off()    
# plt.title('% of Candidates With Division I, II & III',fontdict = {'fontsize': 16,
#         'fontweight': 'bold',
#         'color': 'maroon',}
#         )
ax[0].set_title('% of School Candidates With Division I, II & II',fontdict = {'fontsize': 14,
        'fontweight': 'bold',
        'color': 'black',}
               )
# plt.show()
zenj_results.plot(ax = ax[1], figsize = (15,15), column = 'low_pct', legend = False, cmap = 'Reds',
               k =5, edgecolor = '#ccc', linewidth = 1, 
                 legend_kwds={'label': "% with Division IV & 0",'orientation': "horizontal"}) #scheme = 'quantiles', 
zenj_results.centre.plot(ax = ax[1], color = 'white', markersize = 3000, alpha = 0.5, edgecolor = 'cyan', lw = 2)
for x,y,value  in zip(zenj_results.centre.x, zenj_results.centre.y, zenj_results.low_pct):
    ax[1].annotate(round(value,1), xy=(x,y), xytext = (-10,-4), textcoords = 'offset points', fontsize = 14)    
ax[1].set_axis_off()  

ax[1].set_title('% of School Candidates With Division IV & 0',fontdict = {'fontsize': 14,
        'fontweight': 'bold',
        'color': 'black',})
fig.suptitle('Zanzibar CSEE Results, 2021', fontsize = 16, fontweight = 'bold', color = 'black' )
# plt.savefig('zenj-2021.png', dpi = 300)
# plt.show()

## Part III: GPA Analysis

In [ ]:
import pandas as pd
import glob
import geopandas as gpd
import matplotlib.pyplot as plt

In [ ]:
path = r'G:\My Drive\consultancy\CSEE2022 Analysis\2022' # use your path
all_files = glob.glob(path + "/*.csv")
# print(all_files)
li = []

for filename in all_files:
    fname = filename.split('\\')[-1]
#     print(fname)
    if fname.startswith('school'):
        print('Processing ', fname)
        df = pd.read_csv(filename)

        li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)
frame = frame.drop_duplicates()


frame['CNO'] = frame['name'].apply(lambda x:x.strip().split(' ')[0])
frame

In [ ]:
plt.figure()
plt.scatter(frame.school_gpa, frame.math_gpa, alpha = 0.1, marker = '.', color = 'r', lab)
plt.scatter(frame.school_gpa, frame.swahili_gpa,  alpha = 0.1, marker = '.', color = 'k', lab)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.grid(linestyle = ':')
plt.ylim(0,5.5,)

plt.xlim(0,5.5)

plt.xlabel('School GPA')
plt.ylabel ('Subject GPA')
plt.legend()